In [1]:
import pickle
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score
import matplotlib.pylab as plt
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict, ShuffleSplit
import tsfresh as tf
%matplotlib inline


C:\Users\AOX\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [24]:
df=pd.read_csv('data.csv',header=None)
dfc=df.values

In [31]:
df.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,3,0,0,1,1,654.0,0.65,0.64,-1.12,-0.05,-52.19,74.92,-254.36
1,3,0,0,1,2,892.0,0.66,0.57,-0.77,0.22,-16.69,10.38,-281.18
2,3,0,0,1,3,281.0,0.63,0.42,-0.92,0.11,-46.56,93.08,-280.71
3,3,0,0,1,4,358.0,0.61,0.24,0.84,-0.99,317.01,18.46,236.56
4,3,0,0,1,5,472.0,0.65,-0.48,0.23,-0.69,279.54,103.47,-18.27
5,3,0,0,1,6,7.0,0.33,-0.57,0.03,-0.19,-14.57,-107.82,241.41
6,3,1,0,1,1,664.0,0.98,0.03,-1.84,0.12,-48.55,97.91,-403.20
7,3,1,0,1,2,893.0,1.00,-0.07,-1.25,0.32,1.60,5.92,-437.87
8,3,1,0,1,3,269.0,0.93,-0.44,-1.22,0.29,-55.90,108.08,-440.27
9,3,1,0,1,4,429.0,0.86,-0.20,1.49,-0.85,472.85,36.07,293.73


In [27]:
for i in range(0,len(dfc)):
    dfc[i][10]=dfc[i][10]*math.pi/180
    dfc[i][11]=dfc[i][11]*math.pi/180
    dfc[i][12]=dfc[i][12]*math.pi/180
dfc[0]

array([  3.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   6.54000000e+02,
         6.50000000e-01,   6.40000000e-01,  -1.12000000e+00,
        -5.00000000e-02,  -9.10887337e-01,   1.30760068e+00,
        -4.43941949e+00])

In [28]:
dfc

array([[  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -9.10887337e-01,   1.30760068e+00,  -4.43941949e+00],
       [  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.91295452e-01,   1.81165176e-01,  -4.90751679e+00],
       [  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -8.12625300e-01,   1.62455247e+00,  -4.89931374e+00],
       ..., 
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
          1.29154365e-02,  -1.38893302e+00,   9.49459113e-01],
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
         -1.41912721e+00,   1.09257611e+00,  -6.12436035e-01],
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
         -1.89001705e+00,   1.02415921e+00,  -6.88881456e-01]])

In [ ]:
df1=df[df.columns[0:18]]
df1
numsplits=int(len(df1)/20)
numsplits/6
j=0
count=0
for i in range(0,len(df1)):
    df1.iloc[i][17]=int(j)
    count=count+1
    if count==20:
        j=j+1
        count=0
        if j==6:
            j=0

df1
dff=pd.DataFrame(np.zeros((numsplits,120)))
j=0
gesture=0
for i in range(0, numsplits):
    for coli in range(0,17):
        for k in range(0,7):
            dff.iloc[i][(coli*7)+k] = df1.iloc[(i*20):(i*20)+20][coli].describe()[[k+1]]
    dff.iloc[i][119]=gesture
    gesture=gesture+1
    if gesture==6:
        gesture=0